# Water Filtration Flow Calculation

*Below is the calculation done to ensure that the water filtration system of AutoAquaponics can handle the amount of flow we input without overflowing while ensuring proper solid removal. Below is an image of the entire system plus the grow beds*
***


In [1]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='sys_plus_growbeds.jpg' width='800'></table>"))

## Overall System Description

We will ignore the grow beds from above as they are inconsequential to the overall filter design.

System description:
- 3 filter tanks (each 5 gallons)
- 1 fish tank (100 gallon)
- 1 sump tank (60 gallon)
- 2 grow beds
- Everything is gravity fed with exception of part from pump to grow beds
- Ignore the effect of the ball valve on Overflow 1 as it is full port and assumed to be always open
- **We'll ignore resistance of membrane filtration, flow sensor, and biofilm reactor for now as they are uncertain at the moment**

Goals:
- Maximum input keep HRT of the settling tank at 2 min
- Minimum pipe velocity of lower SLO above V = 1 ft/s to make sure we have enough velocity to lift fish waste up the SLO

We are interested in finding:
- Effective pipe diameters (approximate to nearest standard pipe size) for dotted ($D_{1}$) and solid pipes ($D_{2}$)
- $Z_{1}$, $Z_{2}$, $Z_{3}$ distance between water level of fish tank and filter tanks

[Values of Resistance Coefficient (K) for Various Fitting Types](https://www.plumbingsupply.com/ed-frictionlosses.html)

[Absolute Roughness Coefficient (k) of Different Pipe Materials](https://www.engineeringtoolbox.com/surface-roughness-ventilation-ducts-d_209.html)

In [1]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='whole_sys.jpg' width='800'></table>"))

In [2]:
# Import all libraries and dependencies here
from sympy import symbols, Function, sin, cos, solve, Matrix, simplify, Eq
from sympy.abc import t
import sympy as sym
import pandas as pd
import numpy as np

# This function gives us the resistance coefficient based on diameter and fitting type
def getkL(D, fitting):
    # D is the hydraulic diameter of the pipe in inches
    # fitting is an integer representing a type of PVC fitting as defined in the link below
    # fitting = 0 is Angle Valve LD 55, 1 is Angle Valve LD 150, etc. NaN is inputted for empty values
    # Source: https://www.plumbingsupply.com/ed-frictionlosses.html
    dict = {'0.5':[1.48,4.05,0.08,np.nan,0.22,9.2,2.43,0.48,0.81,0.81,0.43,0.43,np.nan,1.35,0.54,1.62,0.54,0.32,0.32,0.38,0.46,0.65,0.81,0.92,1.03,1.13,1.24,1.35,0.05,0.11,0.22,0.41,0.68,1.09,1.62],
            '0.75': [1.38,3.75,0.08,np.nan,0.2,8.5,2.25,0.45,0.75,0.75,0.4,0.4,np.nan,1.25,0.5,1.5,0.5,0.3,0.3,0.35,0.43,0.6,0.75,0.85,0.95,1.05,1.15,1.25,0.05,0.1,0.2,0.38,0.63,1,1.5],
            '1':[1.27,3.45,0.07,np.nan,0.18,7.8,2.07,0.41,0.69,0.69,0.37,0.37,np.nan,1.15,0.46,1.38,0.46,0.28,0.28,0.32,0.39,0.55,0.69,0.78,0.87,0.97,1.06,1.15,0.05,0.09,0.18,0.35,0.58,0.92,1.38],
            '1.25':[1.21,3.3,0.07,np.nan,0.18,7.5,1.98,0.4,0.66,0.66,0.35,0.35,np.nan,1.1,0.44,1.32,0.44,0.26,0.26,0.31,0.37,0.53,0.66,0.75,0.84,0.92,1.01,1.1,0.04,0.09,0.18,0.33,0.55,0.88,1.32],
            '1.5':[1.16,3.15,0.06,np.nan,0.15,7.1,1.89,0.38,0.63,0.63,0.34,0.34,np.nan,1.05,0.42,1.26,0.42,0.25,0.25,0.29,0.36,0.5,0.63,0.71,0.8,0.88,0.97,1.05,0.04,0.08,0.17,0.32,0.53,0.84,1.26], 
            '2':[1.05,2.85,0.06,0.86,0.15,6.5,1.71,0.34,0.57,0.57,0.3,0.3,np.nan,0.95,0.38,1.14,0.38,0.23,0.23,0.27,0.32,0.46,0.57,0.65,0.72,0.8,0.87,0.95,0.04,0.08,0.15,0.29,0.48,0.76,1.14], 
            '2.5': [0.99,2.7,0.05,0.81,0.14,6.1,1.62,0.32,0.54,0.54,0.29,0.29,np.nan,0.9,0.36,1.08,0.36,0.22,0.22,0.25,0.31,0.43,0.54,0.61,0.68,0.76,0.83,0.9,0.04,0.07,0.14,0.27,0.45,0.72,1.08],
            '3': [0.99,2.7,0.05,0.81,0.14,6.1,1.62,0.32,0.54,0.54,0.29,0.29,np.nan,0.9,0.36,1.08,0.36,0.22,0.22,0.25,0.31,0.43,0.54,0.61,0.68,0.76,0.83,0.9,0.04,0.07,0.14,0.27,0.45,0.72,1.08],
            '4': [0.94,2.55,0.05,0.77,0.14,5.8,1.53,0.31,0.51,0.51,0.27,0.27,np.nan,0.85,0.34,1.02,0.34,0.2,0.2,0.24,0.29,0.41,0.51,0.58,0.65,0.71,0.78,0.85,0.03,0.07,0.14,0.26,0.43,0.68,1.02]}
    df = pd.DataFrame(dict)

    # Choose the closest nominal diameter to the input hydraulic diameter and index the dataframe with it
    lst = [0.5, 0.75, 1, 1.25, 1.5, 2, 2.5, 3, 4]
    D_nom = lst[min(range(len(lst)), key = lambda i: abs(lst[i]-D))]
    return(df[str(D_nom)][fitting])

def getf(Q, D):
    k = 2.30e-5
    D = D/12 #convert to feet
    #A numeric
    A = np.pi*((D/2)**2)
    #V numeric
    V = Q/A
    print("Solve for Reynold's number:")
    Re = V*D/(0.0000108) #kinematic viscosity is in ft^2/s, for water at 20C
    display(Re)
    if Re < 2000:
        print("Reynold's number < 2000, laminar flow:")
        f_num = 64/Re
    elif Re > 2000:
        print("Reynold's number > 2000, turbulent flow:")
        # The below equation is the Moody equation
        # Source: https://www.omnicalculator.com/physics/friction-factor
        f_num = 0.0055 * ( 1 + (2 * (10**4) * k/D + (10**6)/Re)**(1/3))
    display(f_num)
    return f_num

### Part I: Fish Tank to Settling Tank and Membrane Filtration

System description:
- 2 pipe sizes (fish tank to settling tank, everything else)
- 3 distinct velocities (from fish tank to settling tank ($V_{1}$), settling tank overflow to branch input of T-fitting ($V_{2}$), from fish tank to flow input of T-fitting ($V_{3}$), output of T-fitting into membrane filtration tank ($V_{4}$))
 - At $V_{1}$ (this is the only segment with the smaller diameter $D_{1}$, all other segments we are using $D_{2}$) we have:
     - One reentrant entry
     - Two 90 degree elbows
     - One exit
 - At $V_{2}$ we have:
     - One reentrant entry
     - One T-fitting branch
 - At $V_{3}$ we have:
     - One reentrant entry
     - One gate valve
     - Two 90 degree elbows
     - One T-fitting flow
 - At $V_{4}$ we have:
     - One exit
- Maximum input Q = 150 GPH to keep HRT at 2 min
- Minimum pipe velocity V = 1 ft/s to make sure we have enough velocity to lift fish waste up the SLO

We are interested in finding:
- Effective pipe diameters $D_{1}$ and $D_{2}$  (approximate to nearest standard pipe size)
- $Z_{1}$, $Z_{2}$, distances between water level of fish tank and the two filter tanks. $Z_{1}$ just depends on the lower SLO segment while $Z_{2}$ is the larger value between what is needed to drain the settling tank and what is needed to drain the remaining flow from the fish tank through the top SLO

We'll ignore the friction loss from the tanks themselves for now. See image below for a diagram of Part 1:

In [2]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='part1_diag.jpg' width='700'></table>"))

#### Start from lower SLO to settling tank:

In [23]:
# Input variables of interest, change these to adjust system parameters:
totalQ = 1000 #GPH
HRT = 2 #min
D_num = 1.029 #inches
l_num = 6 #feet
filter_V = 5 #gallons
num_90elbow = 2
num_45elbow = 0
num_gatevalve = 0
num_globevalve = 0
num_ballvalve = 1
num_Tflow = 0
num_Tbranch = 0
input_var_list_str = ["Total Volumetric Flow Rate (GPH)", "Desired HRT (min)", "Hydraulic Diameter of PVC Pipe (in)", "Total Pipe Length (ft)", "Settling Tank Volume (gallon)", "# of 90\N{DEGREE SIGN} Elbows", "# of 45\N{DEGREE SIGN} Elbows", "# of Gate Valves", "# of Globe Valves", "# of Ball Valves", "# of T Fittings (Flow)", "# of T Fittings (Branch)"]
input_var_list = [totalQ, HRT, D_num, l_num, filter_V, num_90elbow, num_45elbow, num_gatevalve, num_globevalve, num_ballvalve, num_Tflow, num_Tbranch]

In [5]:
# Define all symbolic variables
Pout, Vout, g, rho, Pin, Vin, Zin, Zout, hin, hs, hL, Patm, Z1, Z2, Z3 = symbols(r'P_out, V_out, g, rho, P_in, V_in, Z_in, Z_out, h_in, h_s, h_L, P_atm, Z_1, Z_2, Z_3') #def all the parameters here
Q, f, f2, f_total, f3, l, l2, l_total, l3, D, D2, V, V2, V3, V4, k, Kl_90elbow, Kl_45elbow, Kl_gatevalve, Kl_globevalve, Kl_ballvalve, Kl_Tflow, Kl_Tbranch, Kl_entry, Kl_exit = symbols(r'Q, f_1, f_2, f_total, f_3, l_1, l_2, l_total, l_3, D, D_2, V_1, V_2, V_3, V_4, k, k_L90elbow, k_L45elbow, k_Lgatevalve, k_Lglobevalve, k_Lballvalve k_Lflow, k_Lbranch, k_Lentry, k_Lexit')

# Define some recurring numeric variables
#A = np.pi*((D/2)**2) #define area in terms of D and pi
filter_V_ft = filter_V*0.133681 #convert filter volume to ft^3

print("Input variables of interest:\n")
for ii in range(len(input_var_list_str)):
    print(input_var_list_str[ii] + ": " + str(input_var_list[ii]))

Input variables of interest:

Total Volumetric Flow Rate (GPH): 1000
Desired HRT (min): 2
Hydraulic Diameter of PVC Pipe (in): 1.029
Total Pipe Length (ft): 6
Settling Tank Volume (gallon): 5
# of 90° Elbows: 2
# of 45° Elbows: 0
# of Gate Valves: 0
# of Globe Valves: 0
# of Ball Valves: 1
# of T Fittings (Flow): 0
# of T Fittings (Branch): 0


#### Start from lower SLO to settling tank:
Assumptions:
- Free surface assumption:
    - Pout = Patm
    - Pin = Patm

- Big vat assumption:
    - Vin = 0
    - Vout = 0

- No pump:
    - hs = 0

In [6]:
# Start with Conservation of Energy
MechEnergyEq = sym.Eq(Pout/(rho*g) + Vout**2/(2*g) + Zout, Pin/(rho*g) + Vin**2/(2*g)+Zin+hs-hL)
print("Start with Conservation of Energy")
display(MechEnergyEq)
soln = solve(MechEnergyEq, Zin-Zout)[0]
print("Symbolic solution for Zin-Zout")
display(Eq(Zin-Zout, soln))

print("According to our assumptions, we can reduce this down to: ")
soln = soln.subs({Pin:Patm, Pout:Patm, Vin:0, Vout:0, hs:0})
display(Eq(Zin-Zout, soln))

Start with Conservation of Energy


Eq(P_out/(g*rho) + V_out**2/(2*g) + Z_out, P_in/(g*rho) + V_in**2/(2*g) + Z_in - h_L + h_s)

Symbolic solution for Zin-Zout


Eq(Z_in - Z_out, (-P_in + P_out + g*rho*(h_L - h_s) + rho*(-V_in**2 + V_out**2)/2)/(g*rho))

According to our assumptions, we can reduce this down to: 


Eq(Z_in - Z_out, h_L)

In [7]:
print("Now calculate head loss (major loss + minor loss)")

majorLoss = (f*l/D)*(V**2)/(2*g)
print("Major loss:")
display(majorLoss)
print("Minor loss:")
minorLoss = (V**2)/(2*g)*(num_90elbow*Kl_90elbow+num_45elbow*Kl_45elbow+num_gatevalve*Kl_gatevalve+num_globevalve*Kl_globevalve+num_ballvalve*Kl_ballvalve+num_Tbranch*Kl_Tbranch+num_Tflow*Kl_Tflow+1*Kl_entry+1*Kl_exit)
display(minorLoss)

print("Substitute in major and minor losses:")
soln1 = simplify(soln.subs(hL, majorLoss+minorLoss))
display(Eq(Zin-Zout, soln1))

Now calculate head loss (major loss + minor loss)
Major loss:


V_1**2*f_1*l_1/(2*D*g)

Minor loss:


V_1**2*(2*k_L90elbow + k_Lballvalve + k_Lentry + k_Lexit)/(2*g)

Substitute in major and minor losses:


Eq(Z_in - Z_out, V_1**2*(D*(2*k_L90elbow + k_Lballvalve + k_Lentry + k_Lexit) + f_1*l_1)/(2*D*g))

In [8]:
#A numeric
A_num = np.pi*(((D_num/12)/2)**2)
print("A_num (ft^2): ", A_num)
#Q numeric
Q_num = filter_V_ft/(HRT*60) #ft^3/s
print("Q_num (GPH): ", Q_num/0.133681*3600)
#V numeric
V_num = Q_num/A_num
print("V_num (ft/s): ", V_num)

#Calculate friction factor
f_num = getf(Q_num, D_num)
display(Eq(f,f_num))
    
print("Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):")
soln1 = soln1.subs({V:V_num, f:f_num})
# Absolute friction value (k) of PVC pipes span from (0.49 - 2.30) 10e-5, so we are going the conservative route
# and using 2.30e-5
# Source: https://www.engineeringtoolbox.com/surface-roughness-ventilation-ducts-d_209.html
# Gravitational constant: 32.17 ft/s^2
# Density of water: 62.4 lbs/ft^3
# Patm = 14.6959 lbs/in^2 * 144 in^2/1 ft^2
soln1 = soln1.subs({Kl_90elbow:getkL(D_num, 9), Kl_45elbow:getkL(D_num, 10), Kl_gatevalve:getkL(D_num, 4), Kl_globevalve:getkL(D_num, 5),
                  Kl_ballvalve:getkL(D_num, 2), Kl_Tflow:getkL(D_num, 13), Kl_Tbranch:getkL(D_num, 14),
                  l:l_num, D:D_num/12, k: 2.30e-5, Kl_entry:0.78, Kl_exit:1, g:32.17, rho:62.4, Patm:14.6959*144})
display(Eq(Z1,soln1*12))

A_num (ft^2):  0.005775081782846648
Q_num (GPH):  150.0
V_num (ft/s):  0.9644957207724746
Solve for Reynold's number:


7657.917412614786

Reynold's number > 2000, turbulent flow:


0.033780508653367916

Eq(f_1, 0.0337805086533679)

Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):


Eq(Z_1, 0.97050135645543)

#### Now figure out $Z_{2}$ through calculating minimum distance needed to drain settling tank to membrane filtration tank

In [9]:
# The remaining Q going through the top SLO
Q_num3 = totalQ*0.133681/3600 - Q_num #in ft^3/s

# Define parameters we can change for this segment
D_num2 = 2.047
l_num2 = 6 #feet
l_num2_wide = 1 #feet
num_90elbow2 = 2
num_45elbow2 = 0
num_gatevalve2 = 0
num_globevalve2 = 0
num_ballvalve2 = 0
num_Tflow2 = 0
num_Tbranch2 = 1
input_var_list_str2 = ["Hydraulic Diameter of PVC Pipe (in)", "Total Pipe Length (ft)", "Settling Tank Volume (gallon)", "# of 90\N{DEGREE SIGN} Elbows", "# of 45\N{DEGREE SIGN} Elbows", "# of Gate Valves", "# of Globe Valves", "# of Ball Valves", "# of T Fittings (Flow)", "# of T Fittings (Branch)"]
input_var_list2 = [D_num2, l_num2, l_num2_wide, num_90elbow2, num_45elbow2, num_gatevalve2, num_globevalve2, num_ballvalve2, num_Tflow2, num_Tbranch2]

print("Input variables of interest (segment 2):\n")
for ii in range(len(input_var_list_str2)):
    print(input_var_list_str2[ii] + ": " + str(input_var_list2[ii]))

Input variables of interest (segment 2):

Hydraulic Diameter of PVC Pipe (in): 2.047
Total Pipe Length (ft): 6
Settling Tank Volume (gallon): 1
# of 90° Elbows: 2
# of 45° Elbows: 0
# of Gate Valves: 0
# of Globe Valves: 0
# of Ball Valves: 0
# of T Fittings (Flow): 0
# of T Fittings (Branch): 1


In [10]:
print("Now calculate head loss (major loss + minor loss)")
majorLoss2 = (f2*l2/D2)*(V2**2)/(2*g) + (f_total*l_total/D2)*(V4**2)/(2*g)
print("Major loss:")
display(majorLoss2)
print("Minor loss:")
minorLoss2 = (V2**2)/(2*g)*(num_90elbow2*Kl_90elbow+num_45elbow2*Kl_45elbow+num_gatevalve2*Kl_gatevalve+num_globevalve2*Kl_globevalve+num_ballvalve2*Kl_ballvalve+num_Tflow2*Kl_Tflow+1*Kl_entry) + (V4**2)/(2*g)*(num_Tbranch2*Kl_Tbranch+1*Kl_exit) 
display(minorLoss2)

print("Substitute in major and minor losses:")
soln2 = simplify(soln.subs(hL, majorLoss2+minorLoss2))
display(Eq(Zin-Zout, soln2))

Now calculate head loss (major loss + minor loss)
Major loss:


V_2**2*f_2*l_2/(2*D_2*g) + V_4**2*f_total*l_total/(2*D_2*g)

Minor loss:


V_2**2*(2*k_L90elbow + k_Lentry)/(2*g) + V_4**2*(k_Lbranch + k_Lexit)/(2*g)

Substitute in major and minor losses:


Eq(Z_in - Z_out, (D_2*(V_2**2*(2*k_L90elbow + k_Lentry) + V_4**2*(k_Lbranch + k_Lexit)) + V_2**2*f_2*l_2 + V_4**2*f_total*l_total)/(2*D_2*g))

In [11]:
# Caclulate A_num2
A_num2 = np.pi*(((D_num2/12)/2)**2)
print("A_num2 (ft^2): ", A_num2)
# Calculate V_num2
V_num2 = Q_num/A_num2
print("V_num2 (ft/s): ", V_num2)

# Calculate two friction factors, one for prior to T (f_num2), one for after T (f_num2_wide)
f_num2 = getf(Q_num, D_num2)
display(Eq(f2,f_num2))

f_num2_wide = getf(Q_num+Q_num3, D_num2)
display(Eq(f_total,f_num2_wide))

A_num2 (ft^2):  0.022854044811468462
V_num2 (ft/s):  0.24372235692263738
Solve for Reynold's number:


3849.5344492333234

Reynold's number > 2000, turbulent flow:


0.04071450945007675

Eq(f_2, 0.0407145094500767)

Solve for Reynold's number:


25663.562994888824

Reynold's number > 2000, turbulent flow:


0.02456678113659133

Eq(f_total, 0.0245667811365913)

In [12]:
print("Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):")
soln2 = soln2.subs({V2:V_num2, V4:(Q_num+Q_num3)/(np.pi*((D_num2/2)**2)), f2:f_num2, f_total:f_num2_wide})
# Absolute friction value (k) of PVC pipes span from (0.49 - 2.30) 10e-5, so we are going the conservative route
# and using 2.30e-5
# Source: https://www.engineeringtoolbox.com/surface-roughness-ventilation-ducts-d_209.html
# Gravitational constant: 32.17 ft/s^2
# Density of water: 62.4 lbs/ft^3
# Patm = 14.6959 lbs/in^2 * 144 in^2/1 ft^2
soln2 = soln2.subs({Kl_90elbow:getkL(D_num2, 9), Kl_45elbow:getkL(D_num2, 10), Kl_gatevalve:getkL(D_num2, 4), Kl_globevalve:getkL(D_num2, 5),
                  Kl_ballvalve:getkL(D_num, 2), Kl_Tflow:getkL(D_num2, 13), Kl_Tbranch:getkL(D_num2, 14),
                  l2:l_num2, l_total:l_num2_wide, D2:D_num2/12, k: 2.30e-5, Kl_entry:0.78, Kl_exit:1, g:32.17, rho:62.4, Patm:14.6959*144})
display(Eq(Z2,soln2*12+soln1*12)) #the actual Z2 here is this value plus Z1 from earlier

Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):


Eq(Z_2, 1.00767429072874)

#### Now look at the flow from the top SLO to the membrane filtration tank to find an alternative value for $Z_{2}$, choose whichever is larger as the final $Z_{2}$

In [13]:
# New input parameters for this segment, change these to adjust system parameters:
l_num3 = 6 #feet
num_90elbow3 = 2
num_45elbow3 = 0
num_gatevalve3 = 0
num_globevalve3 = 0
num_ballvalve3 = 1
num_Tflow3 = 1
num_Tbranch3 = 0
input_var_list_str3 = ["Hydraulic Diameter of PVC Pipe (in)", "Total Pipe Length (ft)", "# of 90\N{DEGREE SIGN} Elbows", "# of 45\N{DEGREE SIGN} Elbows", "# of Gate Valves", "# of Globe Valves", "# of Ball Valves", "# of T Fittings (Flow)", "# of T Fittings (Branch)"]
input_var_list3 = [D_num2, l_num3, num_90elbow3, num_45elbow3, num_gatevalve3, num_globevalve3, num_ballvalve3, num_Tflow3, num_Tbranch3]

print("\nInput variables of interest (segment 3):\n")
for ii in range(len(input_var_list_str3)):
    print(input_var_list_str3[ii] + ": " + str(input_var_list3[ii]))


Input variables of interest (segment 3):

Hydraulic Diameter of PVC Pipe (in): 2.047
Total Pipe Length (ft): 6
# of 90° Elbows: 2
# of 45° Elbows: 0
# of Gate Valves: 0
# of Globe Valves: 0
# of Ball Valves: 1
# of T Fittings (Flow): 1
# of T Fittings (Branch): 0


In [14]:
print("Now calculate head loss (major loss + minor loss)")

majorLoss3 = (f3*l3/D2)*(V3**2)/(2*g) + (f_total*l_total/D2)*(V4**2)/(2*g)
print("Major loss:")
display(majorLoss3)
print("Minor loss:")
minorLoss3 = (V3**2)/(2*g)*(num_90elbow3*Kl_90elbow+num_45elbow3*Kl_45elbow+num_gatevalve3*Kl_gatevalve+num_globevalve3*Kl_globevalve+num_ballvalve3*Kl_ballvalve+num_Tbranch3*Kl_Tbranch+num_Tflow3*Kl_Tflow+1*Kl_entry) + (V4**2)/(2*g)*(num_Tbranch2*Kl_Tbranch+1*Kl_exit)
display(minorLoss3)

print("Substitute in major and minor losses:")
soln3 = simplify(soln.subs(hL, majorLoss3+minorLoss3))
display(Eq(Zin-Zout, soln3))

Now calculate head loss (major loss + minor loss)
Major loss:


V_3**2*f_3*l_3/(2*D_2*g) + V_4**2*f_total*l_total/(2*D_2*g)

Minor loss:


V_3**2*(2*k_L90elbow + k_Lballvalve + k_Lentry + k_Lflow)/(2*g) + V_4**2*(k_Lbranch + k_Lexit)/(2*g)

Substitute in major and minor losses:


Eq(Z_in - Z_out, (D_2*(V_3**2*(2*k_L90elbow + k_Lballvalve + k_Lentry + k_Lflow) + V_4**2*(k_Lbranch + k_Lexit)) + V_3**2*f_3*l_3 + V_4**2*f_total*l_total)/(2*D_2*g))

In [15]:
# Calculate V_num3
V_num3 = Q_num3/A_num2
print("V_num3 (ft/s): ", V_num3)

# Calculate two friction factors, one for prior to T (f_num3), one for after T (f_num2_wide)
f_num3 = getf(Q_num3, D_num2)
display(Eq(f3,f_num3))

f_num2_wide = getf(Q_num+Q_num3, D_num2)
display(Eq(f_total,f_num2_wide))

V_num3 (ft/s):  1.3810933558949452
Solve for Reynold's number:


21814.0285456555

Reynold's number > 2000, turbulent flow:


0.02556282277206841

Eq(f_3, 0.0255628227720684)

Solve for Reynold's number:


25663.562994888824

Reynold's number > 2000, turbulent flow:


0.02456678113659133

Eq(f_total, 0.0245667811365913)

In [16]:
print("Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):")
soln3 = soln3.subs({V3:V_num3, V4:(Q_num+Q_num3)/(np.pi*((D_num2/2)**2)), f3:f_num3, f_total:f_num2_wide})
# Absolute friction value (k) of PVC pipes span from (0.49 - 2.30) 10e-5, so we are going the conservative route
# and using 2.30e-5
# Source: https://www.engineeringtoolbox.com/surface-roughness-ventilation-ducts-d_209.html
# Gravitational constant: 32.17 ft/s^2
# Density of water: 62.4 lbs/ft^3
# Patm = 14.6959 lbs/in^2 * 144 in^2/1 ft^2
soln3 = soln3.subs({Kl_90elbow:getkL(D_num2, 9), Kl_45elbow:getkL(D_num2, 10), Kl_gatevalve:getkL(D_num2, 4), Kl_globevalve:getkL(D_num2, 5),
                  Kl_ballvalve:getkL(D_num, 2), Kl_Tflow:getkL(D_num2, 13), Kl_Tbranch:getkL(D_num2, 14),
                  l3:l_num3, l_total:l_num2_wide, D2:D_num2/12, k: 2.30e-5, Kl_entry:0.78, Kl_exit:1, g:32.17, rho:62.4, Patm:14.6959*144})
display(Eq(Z2,soln3*12))

Final numeric solution for minimal difference in height between the two water levels to prevent overflow (in):


Eq(Z_2, 1.36581165786557)

### Part II: Membrane Filtration to Biofilm Reactor and Sump

System description:
 - 1 pipe size (fish tank to settling tank, everything else)
 - 1 distinct velocity
 - From membrane filtration to biofilm reactor we have:
     - One reentrant entry
     - One 90 degree elbow
     - One exit
 - From biofilm reactor to sump we have:
     - One reentrant entry
     - One 90 degree elbow
     - One exit
- Q = 1000 GPH

We are interested in finding:
- $Z_{3}$, distances between water level of membrane filtration tank and the biofilm reactor tank.

We'll ignore the friction loss from the tanks themselves for now. See image below for a diagram of Part 2:

In [17]:
# Input variables of interest, change these to adjust system parameters:

# Fluid Parameters
Q_num4 = 1000*0.133681/3600 # (ft^3/s)
D_num = 1.029/12  # (ft) IGNORE THIS 
D_num2 = 2.047 # (in)
l_num4 = 1 #feet                 #CHECK WITH BILL

#Pipe Parameters
num_90elbow4 = 2
num_45elbow4 = 0
num_gatevalve4 = 0
num_globevalve4 = 0
num_ballvalve4 = 0
num_Tflow4 = 0
num_Tbranch4 = 0

input_var_list_str = ["Total Volumetric Flow Rate (ft^3/s)", "Hydraulic Diameter of PVC Pipe (ft)", "Total Pipe Length (ft)", "# of 90\N{DEGREE SIGN} Elbows", "# of 45\N{DEGREE SIGN} Elbows", "# of Gate Valves", "# of Globe Valves", "# of Ball Valves", "# of T Fittings (Flow)", "# of T Fittings (Branch)"]
input_var_list = [Q_num4, D_num2, l_num4, num_90elbow4, num_45elbow4, num_gatevalve4, num_globevalve4, num_ballvalve4, num_Tflow4, num_Tbranch4]

print("Input variables of interest:\n")
for ii in range(len(input_var_list_str)):
    print(input_var_list_str[ii] + ": " + str(input_var_list[ii]))

Input variables of interest:

Total Volumetric Flow Rate (ft^3/s): 0.0371336111111111
Hydraulic Diameter of PVC Pipe (ft): 2.047
Total Pipe Length (ft): 1
# of 90° Elbows: 2
# of 45° Elbows: 0
# of Gate Valves: 0
# of Globe Valves: 0
# of Ball Valves: 0
# of T Fittings (Flow): 0
# of T Fittings (Branch): 0


In [18]:
# Define all symbolic variables
Pout, Vout, g, rho, Pin, Vin, Zin, Zout, hin, hs, hL, Patm, Z1, Z2, Z3 = symbols(r'P_out, V_out, g, rho, P_in, V_in, Z_in, Z_out, h_in, h_s, h_L, P_atm, Z_1, Z_2, Z_3') #def all the parameters here
Q, f, f2, f_total, f3, f4, l, l2, l4, l_total, l3, D, D2, V, V2, V3, V4, V5, k, Kl_90elbow, Kl_45elbow, Kl_gatevalve, Kl_globevalve, Kl_ballvalve, Kl_Tflow, Kl_Tbranch, Kl_entry, Kl_exit = symbols(r'Q, f_1, f_2, f_total, f_3, f_4, l_1, l_2, L_4, l_total, l_3, D, D_2, V_1, V_2, V_3, V_4, V_5, k, k_L90elbow, k_L45elbow, k_Lgatevalve, k_Lglobevalve, k_Lballvalve k_Lflow, k_Lbranch, k_Lentry, k_Lexit')

In [19]:
# Start with Conservation of Energy
MechEnergyEq = sym.Eq(Pout/(rho*g) + Vout**2/(2*g) + Zout, Pin/(rho*g) + Vin**2/(2*g)+Zin+hs-hL)
print("Start with Conservation of Energy")
display(MechEnergyEq)

soln4 = solve(MechEnergyEq, Zin-Zout)[0]
print("Symbolic solution for Zin-Zout")
display(Eq(Zin-Zout, soln4))

soln4 = soln4.subs({Pin:Patm, Pout:Patm, Vin:0, Vout:0, hs:0})
print("According to our assumptions, we can reduce this down to: ")
display(Eq(Zin-Zout, soln4))

Start with Conservation of Energy


Eq(P_out/(g*rho) + V_out**2/(2*g) + Z_out, P_in/(g*rho) + V_in**2/(2*g) + Z_in - h_L + h_s)

Symbolic solution for Zin-Zout


Eq(Z_in - Z_out, (-P_in + P_out + g*rho*(h_L - h_s) + rho*(-V_in**2 + V_out**2)/2)/(g*rho))

According to our assumptions, we can reduce this down to: 


Eq(Z_in - Z_out, h_L)

In [20]:
print("Calculation of head loss (major loss + minor loss)")

majorLoss4 = (f4*l4/D2)*(V5**2)/(2*g)
print("Major loss:")
display(majorLoss4)

minorLoss4 = (V5**2)/(2*g)*(num_90elbow4*Kl_90elbow+num_45elbow4*Kl_45elbow+num_gatevalve4*Kl_gatevalve+num_globevalve4*Kl_globevalve+num_ballvalve4*Kl_ballvalve+num_Tflow4*Kl_Tflow+1*Kl_entry) + (V5**2)/(2*g)*(num_Tbranch4*Kl_Tbranch+1*Kl_exit) 
print("Minor loss:")
display(minorLoss4)

soln4 = simplify(soln4.subs(hL, majorLoss4+minorLoss4))
print("Substitute in major and minor losses:")
display(Eq(Zin-Zout, soln4))

Calculation of head loss (major loss + minor loss)
Major loss:


L_4*V_5**2*f_4/(2*D_2*g)

Minor loss:


V_5**2*k_Lexit/(2*g) + V_5**2*(2*k_L90elbow + k_Lentry)/(2*g)

Substitute in major and minor losses:


Eq(Z_in - Z_out, V_5**2*(D_2*(2*k_L90elbow + k_Lentry + k_Lexit) + L_4*f_4)/(2*D_2*g))

In [21]:
# Caclulate A_num2
A_num2 = np.pi*(((D_num2/12)/2)**2)
print("A_num4 (ft^2): ", A_num2)

# Calculate V_num2
V_num5 = Q_num4/A_num2
print("V_num5 (ft/s): ", V_num5)

# Calculate the friction factor
f_num4 = getf(Q_num4, D_num2)
display(Eq(f4,f_num4))

A_num4 (ft^2):  0.022854044811468462
V_num5 (ft/s):  1.6248157128175826
Solve for Reynold's number:


25663.562994888824

Reynold's number > 2000, turbulent flow:


0.02456678113659133

Eq(f_4, 0.0245667811365913)

In [22]:
print("Final Numeric Solution for Difference in Height Between the Two Water Levels to Prevent Overflow (in):")
soln4 = soln4.subs({V5:V_num5, V4:(Q_num4)/(np.pi*((D_num2/2)**2)), f4:f_num4})

soln4 = soln4.subs({Kl_90elbow:getkL(D_num2, 9), Kl_45elbow:getkL(D_num2, 10), Kl_gatevalve:getkL(D_num2, 4), Kl_globevalve:getkL(D_num2, 5),
                  Kl_ballvalve:getkL(D_num, 2), Kl_Tflow:getkL(D_num2, 13), Kl_Tbranch:getkL(D_num2, 14),
                  l4:l_num4, D2:D_num2/12, k: 2.30e-5, Kl_entry:0.78, Kl_exit:1, g:32.17, rho:62.4, Patm:14.6959*144})
display(Eq(Z3,soln4*12))

Final numeric solution for difference in height between the two water levels to prevent overflow (in):


Eq(Z_3, 1.50868815936183)